# **자연어 처리 Exp3 Report**
                                                컴퓨터전자시스템공학부 201603727 허성일
                                                제출일 : 2020-11-12
                                                교수님 : 김낙현 교수님

### 1. 로이터 뉴스 데이터에서 뉴스 주제 46개를 구하는 프로그램을 작성하고, 데이터 1~50까지 각 뉴스의 주제를 구하여 인쇄하라.
*[문제 1]에서 주제를 텍스트로 구하는 것은 내부 데이터로는 해결되지 않으니 주제를 숫자로 적는 것으로 수정함.

### 2. 로이터 뉴스 프로그램을 LSTM을 사용하지 않고 Dense layer를 이용하여 주제를 구하는 프로그램을 작성하라. 이 프로그램의 정확도와 LSTM 버전 프로그램의 정확도를 비교하라.

### 3. 11월2일 수업 자료에서 IMDB 영화평 예측 프로그램에서 Dense 네트워크 구조를 1단으로 하면서 설 숫자를 16, 32, 64개로 했을 때 결과의 정확도가 어떻게 바뀌는지 분석하라.


**1.  
[소스코드]**

In [2]:
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=1000, test_split=0.2)


max_len = 100
X_train = pad_sequences(X_train, maxlen=max_len) # 훈련용 뉴스 기사 패딩
X_test = pad_sequences(X_test, maxlen=max_len) # 테스트용 뉴스 기사 패딩

y_train = to_categorical(y_train) # 훈련용 뉴스 기사 레이블의 원-핫 인코딩
y_test = to_categorical(y_test) # 테스트용 뉴스 기사 레이블의 원-핫 인코딩

model = Sequential()
model.add(Embedding(1000, 120))
model.add(LSTM(120))
model.add(Dense(46, activation='softmax'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

history = model.fit(X_train, y_train, 
                   batch_size=128, 
                   epochs=30, 
                   callbacks=[es, mc], 
                   validation_data=(X_test, y_test))

In [ ]:
result = model.predict(X_test[:50])

In [1]:
for i,x in enumerate(result):
    print("{}번째 기사 주제 : {}".format(i+1,np.argmax(x)),end='\t')
    if i%5==4:print()

NameError: name 'result' is not defined

**[설명]**  
reuters data를 load하고 뉴스 기사를 패딩한다. 각 기사의 레이블은 원-핫 인코딩 하고 lstm 모델
을 작성하여 모델링하고 모델 훈련시킨 다음 x_test 데이터 50개를 예측했다.

**2.  
[소스코드] lstm을 사용하지 않은 모델**

In [ ]:
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=1000, test_split=0.2)


max_len = 100
X_train = pad_sequences(X_train, maxlen=max_len) # 훈련용 뉴스 기사 패딩
X_test = pad_sequences(X_test, maxlen=max_len) # 테스트용 뉴스 기사 패딩

y_train = to_categorical(y_train) # 훈련용 뉴스 기사 레이블의 원-핫 인코딩
y_test = to_categorical(y_test) # 테스트용 뉴스 기사 레이블의 원-핫 인코딩

model = Sequential()
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(46, activation='softmax'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

history = model.fit(X_train, y_train, 
                   batch_size=128, 
                   epochs=30, 
                   callbacks=[es, mc], 
                   validation_data=(X_test, y_test))

**lstm 모델과 평가**  

**model : LSTM모델**

In [ ]:
model.evalutate(X_test,y_test)

**model1 : Dense layer모델**

In [ ]:
model1.evalutate(X_test,y_test)

**3.  
[소스코드]**

sel = 16인 경우

In [1]:
from keras.datasets import imdb
from keras import models 
from keras import layers
import numpy as np

(train_data, train_labels), (test_data, test_labels) = 	imdb.load_data( num_words=10000)

# 입력 텍스트 vectorization
def vectorize_sequences(sequences, dimension=10000): 
    results = np.zeros((len(sequences), dimension)) 
    for i, sequence in enumerate(sequences): 
        results[i, sequence] = 1. 
    return results 

x_train = vectorize_sequences(train_data) 
x_test = vectorize_sequences(test_data)
y_train = np.array(train_labels).astype('float32')
y_test = np.array(test_labels).astype('float32')

model16 = models.Sequential() 
model16.add(layers.Dense(16, activation='relu', input_shape=(10000,))) 
model16.add(layers.Dense(16, activation='relu')) 
model16.add(layers.Dense(1, activation='sigmoid'))

model16.compile(optimizer='rmsprop', loss='binary_crossentropy', 
    metrics=['accuracy'])

model16.fit(x_train, y_train, epochs=4, batch_size=512)
results = model16.evaluate(x_test, y_test)

sel=32인 경우

In [ ]:
model32 = models.Sequential() 
model32.add(layers.Dense(32, activation='relu', input_shape=(10000,))) 
model32.add(layers.Dense(16, activation='relu')) 
model32.add(layers.Dense(1, activation='sigmoid'))

model32.compile(optimizer='rmsprop', loss='binary_crossentropy', 
    metrics=['accuracy'])

model32.fit(x_train, y_train, epochs=4, batch_size=512)
results = model32.evaluate(x_test, y_test)

sel=64인 경우

In [ ]:
model64 = models.Sequential() 
model64.add(layers.Dense(64, activation='relu', input_shape=(10000,))) 
model64.add(layers.Dense(16, activation='relu')) 
model64.add(layers.Dense(1, activation='sigmoid'))

model64.compile(optimizer='rmsprop', loss='binary_crossentropy', 
    metrics=['accuracy'])

model64.fit(x_train, y_train, epochs=4, batch_size=512)
results = model16.evaluate(x_test, y_test)

**[결과]**  
16에서 32로 증가햇을 때는 정확도가 감소하였고 64인 경우 다시 증가하였지만 16보다는 낮은
정확도를 보였다. sel의 수가 많아지면서 모델이 더욱 복잡해지고 overfitting되는 경향을 보여주는 것으로 판단된다.